# * Ex-T Revenue & Subs

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, REMARK, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM (
        SELECT TM_KEY_DAY
            , CASE 	WHEN METRIC_CD LIKE 'TB%S%' 
                    THEN ROW_NUMBER() OVER (PARTITION BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, AREA_CD ORDER BY TM_KEY_DAY DESC)
                    ELSE 1 END FLAG
            , CASE WHEN METRIC_CD LIKE 'TB%S%' THEN 'SUB' ELSE 'REV' END AGG_TYPE
            , METRIC_CD, METRIC_NAME, ACTUAL, COMP_CD, VERSION, AREA_CD, AREA_DESC, AREA_TYPE, LOAD_DATE, REMARK, SRC
        FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
        WHERE REMARK IN ('salisa', 'jutar11')
        AND NOT REGEXP_LIKE(METRIC_CD, '^TCAP|CORP$')
        AND EXISTS (
            SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
            FROM (
                SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            ) TMP
            WHERE TMP_CD = AREA_CD )
        AND TM_KEY_DAY >= {v_start_date} 
    ) TMP
    WHERE FLAG = 1
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, SUBSTR(TM_KEY_DAY,1,6)
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-24, 09:45:06

   -> Summany DataFrame : 427 rows, 17 columns

TDMDBPR : Disconnected


In [3]:
''' Current Month Revenue '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_overall_df = chk_src_df.copy()
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_overall_df = src_t_rev_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_overall_df = src_t_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_overall_df[col] = src_t_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-03-23 04:44:28,20250301,20250320,0,0,"1,629,241,125","1,629,233,724","1,629,233,724","1,622,422,215",0,5,1104,22080
1,TB2R000100,Postpaid Revenue,2025-03-23 04:20:08,20250302,20250322,0,"2,546,270,336","2,546,139,210","2,546,137,455","2,546,137,455",0,0,5,177,1069
2,TB2R010100,Postpaid Revenue B2C,2025-03-23 04:21:05,20250302,20250322,0,"2,290,543,458","2,290,495,985","2,290,494,230","2,290,494,230","2,283,004,367",0,6,1105,6639
3,TB2R020100,Postpaid Revenue B2B,2025-03-23 04:20:46,20250302,20250319,0,"255,726,879","255,643,225","255,643,225","255,643,225",0,0,5,177,1062
4,TB3R000100,Broadband Revenue,2025-03-23 05:10:57,20250302,20250319,0,0,"1,105,343,168","1,105,343,168","1,105,343,168","1,105,426,514",0,5,1102,6608
5,TB4R000100,TVS Revenue,2025-03-19 14:24:58,20250302,20250316,0,0,"117,600,632","117,877,263","117,925,721",0,0,4,176,880


In [4]:
''' Average Revenue per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_avg_mth_df = chk_src_df.copy()
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_avg_mth_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_avg_mth_df[col] = src_t_rev_avg_mth_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_avg_mth_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-03-20 01:42:42,20240101,20250228,0,0,"2,331,116,542","2,331,084,640","2,331,084,640","2,321,354,989",0,5,1104,469200
1,TB2R000100,Postpaid Revenue,2025-03-19 11:39:10,20240102,20250228,0,"3,702,523,438","3,671,427,102","3,671,411,395","3,671,411,395",0,0,5,177,24603
2,TB2R010100,Postpaid Revenue B2C,2025-03-19 11:40:02,20240102,20250228,0,"3,293,619,861","3,287,217,821","3,287,203,341","3,287,204,311","3,262,416,481",0,6,1105,148232
3,TB2R020100,Postpaid Revenue B2B,2025-03-19 11:39:49,20240102,20250228,0,"408,903,578","384,209,282","384,208,054","384,208,054",0,0,5,177,24603
4,TB3R000100,Broadband Revenue,2025-03-21 14:11:16,20240102,20250228,0,0,"1,573,227,163","1,573,118,602","1,573,088,544","1,560,922,362",0,5,1102,138718
5,TB4R000100,TVS Revenue,2025-03-14 13:59:20,20240102,20250228,0,0,"328,867,343","323,172,106","327,520,964","322,352,854",0,5,1104,106301


In [5]:
''' Current Month Subscription '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH']==curr_mth]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-03-23 04:50:12,20250320,20250320,0,0,"17,474,477","17,474,477","17,474,477","17,412,732",0,5,1104,1104
1,TB1S000600,Prepay Active Caller Subs,2025-03-16 04:50:51,20250314,20250314,0,"14,567,698","14,355,888","14,355,888","14,355,888","14,304,384",0,6,1105,1105
2,TB2S000500,Postpaid Active Subs,2025-03-23 05:01:16,20250322,20250322,0,"10,193,643","10,193,098","10,193,086","10,193,086","10,117,996",0,6,1105,1105
3,TB2S010500,Postpaid Active Subs B2C,2025-03-23 05:02:25,20250322,20250322,0,"9,013,477","9,013,230","9,013,218","9,013,218","8,957,963",0,6,1105,1105
4,TB2S020400,Postpaid Active Subs B2B,2025-03-23 04:59:41,20250322,20250322,0,"1,180,166","1,179,868","1,179,868","1,179,868",0,0,5,177,177
5,TB3S000500,Broadband Active Subs,2025-03-23 04:55:27,20250322,20250322,0,0,"3,118,843","3,118,843","3,118,843","3,118,843",0,5,1102,1102
6,TB4S000500,TVS Active Subs,2025-03-23 04:55:39,20250322,20250322,0,0,"1,156,097","1,156,097","1,156,097","1,156,082",0,5,1104,1104


In [6]:
''' Average Subscription per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-03-20 12:12:14,20240131,20250228,0,0,"17,662,374","17,662,181","17,662,181","17,600,486",0,5,1104,15456
1,TB1S000600,Prepay Active Caller Subs,2025-03-07 04:50:40,20240131,20250228,0,"14,441,024","14,342,605","14,362,024","14,362,051","14,312,460",0,6,1105,15470
2,TB2S000500,Postpaid Active Subs,2025-03-19 21:31:43,20240131,20250228,0,"10,021,687","9,921,762","9,922,029","9,922,053","9,849,626",0,6,1105,15470
3,TB2S010500,Postpaid Active Subs B2C,2025-03-19 21:32:10,20240131,20250228,0,"8,891,597","8,887,629","8,887,839","8,887,863","8,832,683",0,6,1105,15470
4,TB2S020400,Postpaid Active Subs B2B,2025-03-19 21:31:15,20240131,20250228,0,"1,130,090","1,034,133","1,034,190","1,034,190",0,0,5,177,2478
5,TB3S000500,Broadband Active Subs,2025-03-01 04:55:28,20240131,20250228,0,0,"3,053,895","3,055,196","3,055,196","3,055,841",0,5,1102,15412
6,TB4S000500,TVS Active Subs,2025-03-01 04:55:38,20240131,20250228,0,0,"1,220,146","1,219,989","1,219,989","1,307,223",0,5,8408,22760


In [7]:
''' METRIC Summary '''

src_t_rev_mth_df = chk_src_df.copy()
src_t_rev_mth_df = src_t_rev_mth_df.groupby(['REMARK', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD']).reset_index()

mod_col_list = src_t_rev_mth_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_rev_mth_df.to_string(max_cols=10)}')


     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE  START_DAY   END_DAY                  P  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-23 04:44:28   20240101  20250320                  0              5         1104   491280
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-23 04:41:29   20240101  20250320                  0              4          176    78320
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-23 04:43:00   20240101  20250320                  0              4          176    78320
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-23 04:50:12   20240131  20250320                  0              5         1104    16560
4   jutar11     TB1S000600   Prepay Active Caller Subs 2025-03-16 04:50:51   20240131  20250314        216,742,036              6         1105    16575
5   jutar11     TB2R000100            Postpaid Revenue 2025-03-23 04:20:08   20240102  

In [8]:
''' METRIC Current Month '''

# v_tm_key_mth = '202502'
curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_t_rev_mth_df_cols = ['REMARK', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_mth: {curr_mth}')

src_t_rev_mth_df = chk_src_df[src_t_rev_mth_df_cols].copy()
src_t_rev_mth_df = src_t_rev_mth_df.loc[src_t_rev_mth_df['TM_KEY_MTH']==curr_mth]
src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

mod_col_list = src_t_rev_mth_df.iloc[:, 7:14].columns.tolist()
for col in mod_col_list:
    src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_rev_mth_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_mth: 202503

     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE TM_KEY_MTH  START_DAY   END_DAY  C              P              G              H             HH           CCAA CCAATT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-23 04:44:28     202503   20250301  20250320  0              0  1,629,241,125  1,629,233,724  1,629,233,724  1,622,422,215      0
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-23 04:41:29     202503   20250301  20250320  0              0    205,494,546    205,494,546    205,494,546              0      0
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-23 04:43:00     202503   20250301  20250320  0              0  1,423,746,580  1,423,739,179  1,423,739,179              0      0
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-23 04:50:12     202503   20250320  20250320  0              0     17,474,477     17,474,477     17,474,477     17,412,73

## Actual : Transaction
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [9]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+ PARALLEL(8)*/ 
         REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    --WHERE METRIC_CD = 'TB3R000100' --TOL Revenue
    WHERE METRIC_CD IN (
        'TB1R000100' --'Prepay Revenue
        , 'TB2R010100' --Postpaid Revenue B2C
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         )
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Daily DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101



TDMDBPR : Connected

   -> Execute query... 2025-03-24, 09:46:18

   -> Daily DataFrame : 153 rows, 12 columns

TDMDBPR : Disconnected


In [10]:
''' METRIC by Day '''

v_tm_key_day = 20250101
# src_t_rev_day_df = ['SRC', 'REMARK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT', 'LOAD_DATE']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

# src_t_rev_day_df = chk_src_df[src_t_rev_day_df_cols].copy()
src_t_rev_day_df = chk_src_df.copy()
src_t_rev_day_df = src_t_rev_day_df.loc[src_t_rev_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_t_rev_day_df = src_t_rev_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_t_rev_day_df.iloc[:, 4:11].columns.tolist()
for col in mod_col_list:
    src_t_rev_day_df[col] = src_t_rev_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_t_rev_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_t_rev_day_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_t_rev_day_df...

   -> DataFrame : 153 rows, 12 columns


In [11]:
''' TB1R000100 : Prepay Revenue '''

v_metric_cd = 'TB1R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB1R000100 : Prepay Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1R000100_day_df = chk_src_df.copy()
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['METRIC_CD']==v_metric_cd]
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB1R000100_day_df = TB1R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB1R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB1R000100_day_df[col] = TB1R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1R000100_day_df


TB1R000100 : Prepay Revenue...

   -> v_metric_cd: TB1R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB1R000100,Prepay Revenue,20250301,2025-03-23 04:44:28,0,0,"98,893,010","98,892,026","98,892,026","98,436,691",0
1,jutar11,TB1R000100,Prepay Revenue,20250302,2025-03-23 04:44:28,0,0,"97,914,184","97,913,942","97,913,942","97,449,214",0
2,jutar11,TB1R000100,Prepay Revenue,20250303,2025-03-23 04:44:28,0,0,"97,337,855","97,337,329","97,337,329","96,894,655",0
3,jutar11,TB1R000100,Prepay Revenue,20250304,2025-03-23 04:44:28,0,0,"87,086,243","87,085,765","87,085,765","86,700,087",0
4,jutar11,TB1R000100,Prepay Revenue,20250305,2025-03-23 04:44:28,0,0,"88,604,071","88,603,914","88,603,914","88,250,640",0
5,jutar11,TB1R000100,Prepay Revenue,20250306,2025-03-23 04:44:28,0,0,"83,013,674","83,013,547","83,013,547","82,665,697",0
6,jutar11,TB1R000100,Prepay Revenue,20250307,2025-03-23 04:44:28,0,0,"84,160,537","84,159,305","84,159,305","83,816,147",0
7,jutar11,TB1R000100,Prepay Revenue,20250308,2025-03-23 04:44:28,0,0,"79,923,823","79,923,741","79,923,741","79,591,388",0
8,jutar11,TB1R000100,Prepay Revenue,20250309,2025-03-23 04:44:28,0,0,"77,199,127","77,199,077","77,199,077","76,884,341",0
9,jutar11,TB1R000100,Prepay Revenue,20250310,2025-03-23 04:44:28,0,0,"83,599,867","83,599,749","83,599,749","83,263,207",0


In [12]:
''' TB2R010100 : Postpaid Revenue B2C '''

v_metric_cd = 'TB2R010100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB2R010100 : Postpaid Revenue B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB2R010100_day_df = chk_src_df.copy()
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['METRIC_CD']==v_metric_cd]
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB2R010100_day_df = TB2R010100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB2R010100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB2R010100_day_df[col] = TB2R010100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB2R010100_day_df


TB2R010100 : Postpaid Revenue B2C...

   -> v_metric_cd: TB2R010100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB2R010100,Postpaid Revenue B2C,20250302,2025-03-23 04:21:05,0,"475,139,708","475,131,519","475,130,620","475,130,620","473,626,578",0
1,jutar11,TB2R010100,Postpaid Revenue B2C,20250306,2025-03-23 04:21:05,0,"279,715,610","279,709,697","279,709,697","279,709,697","278,392,277",0
2,jutar11,TB2R010100,Postpaid Revenue B2C,20250310,2025-03-23 04:21:05,0,"441,775,167","441,766,179","441,766,179","441,766,179","440,499,546",0
3,jutar11,TB2R010100,Postpaid Revenue B2C,20250313,2025-03-23 04:21:05,0,"388,893,015","388,883,919","388,883,470","388,883,470","387,740,186",0
4,jutar11,TB2R010100,Postpaid Revenue B2C,20250316,2025-03-23 04:21:05,0,"353,424,419","353,419,418","353,419,011","353,419,011","352,319,921",0
5,jutar11,TB2R010100,Postpaid Revenue B2C,20250319,2025-03-23 04:21:05,0,"351,595,416","351,585,130","351,585,130","351,585,130","350,425,735",0
6,jutar11,TB2R010100,Postpaid Revenue B2C,20250322,2025-03-23 04:21:05,0,123,123,123,123,123,0


In [13]:
''' TB3R000100 : Broadband Revenue '''

v_metric_cd = 'TB3R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB3R000100 : Broadband Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3R000100_day_df = chk_src_df.copy()
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['METRIC_CD']==v_metric_cd]
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3R000100_day_df = TB3R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3R000100_day_df[col] = TB3R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3R000100_day_df


TB3R000100 : Broadband Revenue...

   -> v_metric_cd: TB3R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB3R000100,Broadband Revenue,20250302,2025-03-23 05:10:57,0,0,"218,003,954","218,003,954","218,003,954","218,039,329",0
1,salisa,TB3R000100,Broadband Revenue,20250306,2025-03-23 05:10:57,0,0,"143,296,128","143,296,128","143,296,128","143,296,128",0
2,salisa,TB3R000100,Broadband Revenue,20250310,2025-03-23 05:10:57,0,0,"232,920,987","232,920,987","232,920,987","232,946,249",0
3,salisa,TB3R000100,Broadband Revenue,20250313,2025-03-23 05:10:57,0,0,"173,102,265","173,102,265","173,102,265","173,102,265",0
4,salisa,TB3R000100,Broadband Revenue,20250316,2025-03-23 05:10:57,0,0,"174,101,735","174,101,735","174,101,735","174,105,080",0
5,salisa,TB3R000100,Broadband Revenue,20250319,2025-03-23 05:10:57,0,0,"163,918,098","163,918,098","163,918,098","163,937,462",0


In [14]:
''' TB4R000100 : TVS Revenue '''

v_metric_cd = 'TB4R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB4R000100 : TVS Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4R000100_day_df = chk_src_df.copy()
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['METRIC_CD']==v_metric_cd]
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4R000100_day_df = TB4R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4R000100_day_df[col] = TB4R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4R000100_day_df


TB4R000100 : TVS Revenue...

   -> v_metric_cd: TB4R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB4R000100,TVS Revenue,20250302,2025-03-19 14:24:58,0,0,"33,915,217","33,928,404","33,945,767",0,0
1,salisa,TB4R000100,TVS Revenue,20250306,2025-03-19 14:24:58,0,0,"9,508,034","9,599,873","9,605,668",0,0
2,salisa,TB4R000100,TVS Revenue,20250310,2025-03-19 14:24:58,0,0,"29,561,765","29,671,805","29,689,081",0,0
3,salisa,TB4R000100,TVS Revenue,20250313,2025-03-19 14:24:58,0,0,"21,702,020","21,763,584","21,771,609",0,0
4,salisa,TB4R000100,TVS Revenue,20250316,2025-03-19 14:24:58,0,0,"22,913,596","22,913,596","22,913,596",0,0
